Exercise. Modify the code so that:
    
    The nodes don't move. 

    Instead, after we select a node, we calculate a radius of a nbh and then we change the color (weight) of the  fractal's pixels in that nbh. 
    The code should have just a couple of iterations.
    Each iterations only 4 nodes are selected randomly.
    

In [121]:

%matplotlib inline

import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import cmath
from copy import copy

class node():
    """The points consist of a coordinate and the color."""
    def __init__(self,x,y,v):
        self.c = x+y*1j
        self.v = v


def mandelbrot( h,w, maxit=500 ):
    """Returns an image of a variant of the Mandelbrot fractal of size (h,w)."""
    y,x = np.ogrid[ -1.5:1.5:h*1j, -1:2:w*1j ]
    c = x+y*1j
    z = c
    divtime = maxit + np.zeros(z.shape, dtype=int)

    for i in tqdm_notebook(range(maxit)):
        z = z**5 + c
        diverge = z*np.conj(z) > 2**2            # who is diverging
        div_now = diverge & (divtime==maxit)  # who is diverging now
        divtime[div_now] = i                  # note when
        z[diverge] = 2                        # avoid diverging too much

    return divtime


Mandel = mandelbrot(500,500)

fig = plt.figure()

ax = fig.add_axes([0., 0., 1., 1., ])
# Hide grid lines
ax.grid(False)
ax.axis('off')
# Hide axes ticks

plt.imshow(Mandel)
plt.show()


###Here we define the auxiliary functions        
def decay(iteration,totalIterations):
    return np.exp(-iteration/totalIterations)

def findLocalNbh(Sigma):# a local NBH of nodes[indexX][indexY] of radius sigma
    good = set()
    sigma=int(Sigma)+1
    for x in range(0,sigma+1):
        for y in range(0,sigma+1):
            if abs(x+y*1j)<sigma:
                good.add(x+y*1j)            
                good.add(-x+y*1j)            
                good.add(x+-y*1j)            
                good.add(-x+-y*1j)
    return good
def findNbh(indexX,indexY,nodes, localNbh):# a local NBH of nodes[indexX][indexY] of radius sigma
    localList = list(localNbh)
    newNbh = [nodes[indexX][indexY].c+z for z in  localList if limits(nodes[indexX][indexY].c+z)[1]]
    return newNbh
    
def findBMU(indexX,indexY,nodes):# the node that has closest value
    return nodes[indexX][indexY]

def limits(z):# we have a finite canvas[ -1.5:1.5:h*1j, -1:2:w*1j ]
        valReal = 499
    elif z.real<0:
        valReal = 0
    else:
        valReal = z.real
    if z.imag>499:
        valImag = 499
    elif z.imag<0:
        valImag = 0
    else:
        valImag = z.imag
    return valReal+valImag*1j, z == valReal+valImag*1j

### We are going to select nodes on the Mandelbrot set
nodes = np.zeros((6,6),dtype=object)
for x_values in range(2,8):
    for y_values in range(2,8):
        nodes[x_values-2][y_values-2]=node(50*x_values,50*y_values,Mandel[50*x_values,50*y_values])
        
fig = plt.figure()

ax = fig.add_axes([0., 0., 1., 1., ])
# Hide grid lines
ax.grid(False)
ax.axis('off')
# Hide axes ticks

plt.imshow(Mandel)

copyNodes = np.ravel(nodes)
for mark in tqdm_notebook(copyNodes):
    plt.scatter(mark.c.real, mark.c.imag, s=5, c='red', marker='o')

plt.show()
### parameters
numberOfIterations = 3
sigma_0=70
L_0=10
Julia2 = copy(Mandel)

for iteration in tqdm_notebook(range(numberOfIterations)):
    dec = decay(iteration,numberOfIterations/np.log(sigma_0))
    dist = sigma_0*dec
    print(dist, ' dist')
    nbh = findLocalNbh(dist) # returns a set
    print('--In iteration ', iteration,' we have radius ',dist)
    
    fig = plt.figure()

    ax = fig.add_axes([0., 0., 1., 1., ])
    # Hide grid lines
    ax.grid(False)
    ax.axis('off')
    # Hide axes ticks
    
    for i in range(5):
        toChange = []
        x_values = np.random.randint(0,6)
        y_values = np.random.randint(0,6)
        BMUc = nodes[x_values][y_values].c
        region = findNbh(x_values,y_values,nodes, nbh)
        for nearNodes in region:
            _, ef  =limits( nearNodes + L_0*(BMUc-nearNodes))
            if ef:
                toChange.append(nearNodes)#I am working with index 0-500 or with actual coordinates?
        for mark in toChange:
            point = Julia2[int(mark.real)][int(mark.imag)] 
            Julia2[int(mark.real)][int(mark.imag)]=point+L_0*dec*abs(mark-BMUc)/dist**2*(Julia2[x_values][y_values]-point)
        print(len(toChange), toChange[-1], dec,abs(mark-BMUc)/dist**2)
    plt.imshow(Julia2)
    plt.show()


IndentationError: unindent does not match any outer indentation level (<ipython-input-121-553e3bd491e2>, line 73)

In [120]:
Mandel[5][5]

0